In [ ]:
# ============================================
# 04_quant_signals.ipynb - Señales cuantitativas
# ============================================

!pip -q install pandas pyarrow

import os
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/drive', force_remount=False)

BASE = "/content/drive/MyDrive/investment_ai"
DIRS = {
    "reports": f"{BASE}/reports",
    "clean": f"{BASE}/data/clean"
}

# --- 1. Leer cartera enriquecida (para obtener tickers activos) ---
ENRICHED_PATH = f"{DIRS['reports']}/portfolio_enriched_final.csv"
if not os.path.exists(ENRICHED_PATH):
    raise FileNotFoundError("Ejecuta primero 02_portfolio_exposure.ipynb")

pf = pd.read_csv(ENRICHED_PATH)
# Obtener tickers únicos (excluir cash y otros sin ticker)
active_tickers = pf[pf["ticker_yf"].notna() & (pf["ticker_yf"] != "-")]["ticker_yf"].unique().tolist()
print(f"🔍 Tickers activos: {active_tickers}")

# --- 2. Leer precios limpios ---
PRICES_PATH = f"{DIRS['clean']}/etfs_prices_clean.parquet"
INDICES_PATH = f"{DIRS['clean']}/indices_prices_clean.parquet"
FX_PATH = f"{DIRS['clean']}/fx_rates_clean.parquet"

# Combinar todos los precios disponibles
all_prices = pd.DataFrame()

if os.path.exists(PRICES_PATH):
    etfs = pd.read_parquet(PRICES_PATH)
    all_prices = pd.concat([all_prices, etfs], axis=1)

if os.path.exists(INDICES_PATH):
    indices = pd.read_parquet(INDICES_PATH)
    all_prices = pd.concat([all_prices, indices], axis=1)

if os.path.exists(FX_PATH):
    fx = pd.read_parquet(FX_PATH)
    all_prices = pd.concat([all_prices, fx], axis=1)

# Asegurar que los tickers de la cartera estén en los precios
available_tickers = [t for t in active_tickers if t in all_prices.columns]
print(f"✅ Tickers con precios: {available_tickers}")

# --- 3. Leer régimen de mercado ---
DAILY_PATH = f"{DIRS['reports']}/portfolio_daily_value.csv"
regime = "calma"  # por defecto

if os.path.exists(DAILY_PATH):
    daily = pd.read_csv(DAILY_PATH, index_col=0, parse_dates=True)
    current_dd = daily["drawdown"].min()
    current_vol = daily["valor_mejorado"].pct_change().std() * np.sqrt(252)

    if current_dd < -0.08:
        regime = "estrés"
    elif current_vol > 0.15:
        regime = "mixto"
    else:
        regime = "calma"

print(f"📊 Régimen de mercado: {regime}")

# --- 4. Calcular señales por activo ---
signals_list = []

for ticker in available_tickers:
    if ticker not in all_prices.columns:
        continue

    price_series = all_prices[ticker].dropna()
    if len(price_series) < 252:
        continue

    # --- Momentum (12-1) ---
    ret_12m = price_series.pct_change(252).iloc[-1]
    ret_1m = price_series.pct_change(21).iloc[-1]
    momentum = ret_12m - ret_1m
    momentum_signal = "positivo" if momentum > 0 else "negativo"

    # --- Low volatility ---
    vol_6m = price_series.pct_change().tail(126).std() * np.sqrt(252)
    # Comparar con percentil del universo
    all_vols = []
    for t in available_tickers:
        if t in all_prices.columns:
            v = all_prices[t].pct_change().tail(126).std() * np.sqrt(252)
            if pd.notna(v):
                all_vols.append(v)
    if all_vols:
        vol_percentile = (np.array(all_vols) < vol_6m).mean()
        low_vol = vol_percentile < 0.3  # top 30% menos volátil
    else:
        low_vol = False

    # --- Señal final ---
    if regime == "estrés":
        if momentum_signal == "negativo":
            signal = "Reducir"
        else:
            signal = "Mantener"
    elif regime == "calma":
        if momentum_signal == "positivo" and low_vol:
            signal = "Aumentar"
        else:
            signal = "Mantener"
    else:  # mixto
        signal = "Mantener"

    signals_list.append({
        "Activo": ticker,
        "Régimen": regime,
        "Momentum": momentum_signal,
        "LowVol": low_vol,
        "Volatilidad_6m_%": vol_6m,
        "Señal": signal
    })

# --- 5. Mostrar resultados ---
if signals_list:
    signals_df = pd.DataFrame(signals_list)
    print("=== 📈 SEÑALES CUANTITATIVAS ===")
    display(signals_df)

    # Guardar
    signals_df.to_csv(f"{DIRS['reports']}/quant_signals.csv", index=False)
    print(f"\n✅ Señales guardadas en: {DIRS['reports']}/quant_signals.csv")
else:
    print("⚠️ No se generaron señales. Revisa los tickers y precios.")

# --- 6. Resumen ejecutivo ---
if signals_list:
    summary = signals_df["Señal"].value_counts().to_dict()
    print("\n=== 📊 RESUMEN DE SEÑALES ===")
    for action, count in summary.items():
        print(f"- {action}: {count} activos")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔍 Tickers activos: ['0P00000LRT.F', '0P00000F24.F', '0P0001FAMF.F', '0P0001K6GO.F', '0P000125YU.L', 'IE0007472115.IR', '0P0001CJGN.F', 'EIMI.L', 'EUN2.DE', 'INRG.MI', 'DFNS.MI', 'FOO', 'ZEG', 'ASML', '5DQ2y', 'ACN']
✅ Tickers con precios: ['0P00000LRT.F', '0P00000F24.F', '0P0001FAMF.F', '0P0001K6GO.F', '0P000125YU.L', 'IE0007472115.IR', '0P0001CJGN.F', 'EIMI.L', 'EUN2.DE', 'INRG.MI', 'DFNS.MI']
📊 Régimen de mercado: calma
=== 📈 SEÑALES CUANTITATIVAS ===


/tmp/ipython-input-3029436089.py:95: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  v = all_prices[t].pct_change().tail(126).std() * np.sqrt(252)


,Activo,Régimen,Momentum,LowVol,Volatilidad_6m_%,Señal
0,0P00000LRT.F,calma,positivo,True,0.001452,Aumentar
1,0P00000F24.F,calma,positivo,True,0.001469,Aumentar
2,0P0001FAMF.F,calma,positivo,True,0.008949,Aumentar
3,0P0001K6GO.F,calma,positivo,False,0.020317,Mantener
4,0P000125YU.L,calma,positivo,True,0.013045,Aumentar
5,IE0007472115.IR,calma,positivo,False,0.036155,Mantener
6,0P0001CJGN.F,calma,positivo,False,0.166587,Mantener
7,EIMI.L,calma,positivo,False,0.183558,Mantener
8,EUN2.DE,calma,positivo,False,0.192292,Mantener
9,INRG.MI,calma,negativo,False,0.214180,Mantener



✅ Señales guardadas en: /content/drive/MyDrive/investment_ai/reports/quant_signals.csv

=== 📊 RESUMEN DE SEÑALES ===
- Mantener: 7 activos
- Aumentar: 4 activos


In [1]:
# ============================================
# 04_quant_signals.ipynb - Señales cuantitativas (MEJORADO + CONTRATO)
# ============================================

import os
import pandas as pd
import numpy as np
import json
from google.colab import drive

# Instalación segura
try:
    import pandas as pd
except ImportError:
    !pip -q install pandas pyarrow
    import pandas as pd

drive.mount('/content/drive', force_remount=False)

BASE = "/content/drive/MyDrive/investment_ai"
DIRS = {
    "reports": f"{BASE}/reports",
    "clean": f"{BASE}/data/clean"
}

# --- 1. Leer cartera enriquecida (para obtener tickers activos) ---
ENRICHED_PATH = f"{DIRS['reports']}/portfolio_enriched_final.csv"
if not os.path.exists(ENRICHED_PATH):
    raise FileNotFoundError("❌ Ejecuta primero 02_portfolio_exposure.ipynb")

pf = pd.read_csv(ENRICHED_PATH)
# Obtener tickers únicos (excluir cash y otros sin ticker)
active_tickers = pf[pf["ticker_yf"].notna() & (pf["ticker_yf"] != "-")]["ticker_yf"].unique().tolist()
print(f"🔍 Tickers activos: {active_tickers}")

if not active_tickers:
    raise ValueError("❌ No hay tickers activos en la cartera.")

# --- 2. Leer precios limpios ---
PRICES_PATH = f"{DIRS['clean']}/etfs_prices_clean.parquet"
INDICES_PATH = f"{DIRS['clean']}/indices_prices_clean.parquet"
FX_PATH = f"{DIRS['clean']}/fx_rates_clean.parquet"

# Combinar todos los precios disponibles
all_prices = pd.DataFrame()

files_found = []
if os.path.exists(PRICES_PATH):
    etfs = pd.read_parquet(PRICES_PATH)
    all_prices = pd.concat([all_prices, etfs], axis=1)
    files_found.append("etfs")

if os.path.exists(INDICES_PATH):
    indices = pd.read_parquet(INDICES_PATH)
    all_prices = pd.concat([all_prices, indices], axis=1)
    files_found.append("índices")

if os.path.exists(FX_PATH):
    fx = pd.read_parquet(FX_PATH)
    all_prices = pd.concat([all_prices, fx], axis=1)
    files_found.append("FX")

if all_prices.empty:
    raise FileNotFoundError("❌ No se encontraron archivos de precios válidos.")

print(f"✅ Archivos de precios cargados: {', '.join(files_found)}")
print(f"✅ Precios disponibles para {len(all_prices.columns)} instrumentos")

# Asegurar que los tickers de la cartera estén en los precios
available_tickers = [t for t in active_tickers if t in all_prices.columns]
if not available_tickers:
    raise ValueError("❌ Ningún ticker activo tiene datos de precios.")

print(f"✅ Tickers con precios: {available_tickers}")

# --- 3. Leer régimen de mercado ---
DAILY_PATH = f"{DIRS['reports']}/portfolio_daily_value.csv"
regime = "calma"  # por defecto

if os.path.exists(DAILY_PATH):
    try:
        daily = pd.read_csv(DAILY_PATH, index_col=0, parse_dates=True)
        if "drawdown" in daily.columns and "valor_mejorado" in daily.columns:
            current_dd = daily["drawdown"].min()
            current_vol = daily["valor_mejorado"].pct_change().std() * np.sqrt(252)

            if current_dd < -0.08:
                regime = "estrés"
            elif current_vol > 0.15:
                regime = "mixto"
            else:
                regime = "calma"
        print(f"📊 Régimen de mercado: {regime} (DD: {current_dd:.1%}, Vol: {current_vol:.1%})")
    except Exception as e:
        print(f"⚠️ Error al leer régimen: {e}. Asumiendo 'calma'.")
else:
    print("⚠️ portfolio_daily_value.csv no encontrado. Asumiendo régimen 'calma'.")

# --- 4. Calcular señales por activo ---
signals_list = []

# Pre-calcular volatilidades para todos los tickers disponibles (eficiencia)
vol_6m_dict = {}
for ticker in available_tickers:
    price_series = all_prices[ticker].dropna()
    if len(price_series) >= 126:
        vol = price_series.pct_change().tail(126).std() * np.sqrt(252)
        if pd.notna(vol):
            vol_6m_dict[ticker] = vol

# Calcular percentil de volatilidad
all_vols = list(vol_6m_dict.values())
vol_percentiles = {}
if all_vols:
    all_vols_arr = np.array(all_vols)
    for ticker, vol in vol_6m_dict.items():
        vol_percentiles[ticker] = (all_vols_arr < vol).mean()
else:
    vol_percentiles = {t: 0.5 for t in available_tickers}

for ticker in available_tickers:
    price_series = all_prices[ticker].dropna()
    if len(price_series) < 252:
        print(f"⚠️ {ticker}: datos insuficientes para momentum (<252 días)")
        continue

    # --- Momentum (12-1) ---
    ret_12m = price_series.pct_change(252).iloc[-1]
    ret_1m = price_series.pct_change(21).iloc[-1]
    momentum = ret_12m - ret_1m
    momentum_signal = "positivo" if momentum > 0 else "negativo"

    # --- Low volatility ---
    vol_6m = vol_6m_dict.get(ticker, np.nan)
    vol_percentile = vol_percentiles.get(ticker, 0.5)
    low_vol = vol_percentile < 0.3  # top 30% menos volátil

    # --- Señal final ---
    if regime == "estrés":
        if momentum_signal == "negativo":
            signal = "Reducir"
        else:
            signal = "Mantener"
    elif regime == "calma":
        if momentum_signal == "positivo" and low_vol:
            signal = "Aumentar"
        else:
            signal = "Mantener"
    else:  # mixto
        signal = "Mantener"

    signals_list.append({
        "Activo": ticker,
        "Régimen": regime,
        "Momentum": momentum_signal,
        "LowVol": low_vol,
        "Volatilidad_6m_%": vol_6m,
        "Señal": signal
    })

# --- 5. Mostrar resultados ---
if signals_list:
    signals_df = pd.DataFrame(signals_list)
    print("=== 📈 SEÑALES CUANTITATIVAS ===")
    print(signals_df.to_string(index=False))

    # Guardar CSV
    CSV_PATH = f"{DIRS['reports']}/quant_signals.csv"
    signals_df.to_csv(CSV_PATH, index=False)
    print(f"\n✅ Señales guardadas en: {CSV_PATH}")

    # Guardar resumen para orquestador
    señales_dict = {row["Activo"]: row["Señal"] for _, row in signals_df.iterrows()}
    resumen = signals_df["Señal"].value_counts().to_dict()

    quant_summary = {
        "fecha": pd.Timestamp.now().strftime("%Y-%m-%d"),
        "regimen": regime,
        "tickers_analizados": available_tickers,
        "señales": señales_dict,
        "resumen": resumen
    }

    JSON_PATH = f"{DIRS['reports']}/quant_signals_latest.json"
    with open(JSON_PATH, "w") as f:
        json.dump(quant_summary, f, indent=2)
    print(f"✅ Resumen para orquestador: {JSON_PATH}")

    # --- 6. Resumen ejecutivo ---
    print("\n=== 📊 RESUMEN DE SEÑALES ===")
    for action, count in sorted(resumen.items()):
        print(f"- {action}: {count} activos")
else:
    print("⚠️ No se generaron señales. Revisa los tickers y precios.")
    # Guardar JSON vacío
    quant_summary = {
        "fecha": pd.Timestamp.now().strftime("%Y-%m-%d"),
        "regimen": regime,
        "tickers_analizados": [],
        "señales": {},
        "resumen": {}
    }
    JSON_PATH = f"{DIRS['reports']}/quant_signals_latest.json"
    with open(JSON_PATH, "w") as f:
        json.dump(quant_summary, f, indent=2)
    print(f"✅ Resumen vacío guardado: {JSON_PATH}")

Mounted at /content/drive
🔍 Tickers activos: ['0P00000LRT.F', '0P00000F24.F', '0P0001FAMF.F', '0P0001K6GO.F', '0P000125YU.L', 'IE0007472115.IR', '0P0001CJGN.F', 'EIMI.L', 'EUN2.DE', 'INRG.MI', 'DFNS.MI', 'FOO', 'ZEG', 'ASML', '5DQ2y', 'ACN']
✅ Archivos de precios cargados: etfs, índices, FX
✅ Precios disponibles para 18 instrumentos
✅ Tickers con precios: ['0P00000LRT.F', '0P00000F24.F', '0P0001FAMF.F', '0P0001K6GO.F', '0P000125YU.L', 'IE0007472115.IR', '0P0001CJGN.F', 'EIMI.L', 'EUN2.DE', 'INRG.MI', 'DFNS.MI']
📊 Régimen de mercado: calma (DD: -3.8%, Vol: 3.5%)
=== 📈 SEÑALES CUANTITATIVAS ===
         Activo Régimen Momentum  LowVol  Volatilidad_6m_%    Señal
   0P00000LRT.F   calma positivo    True          0.001452 Aumentar
   0P00000F24.F   calma positivo    True          0.001469 Aumentar
   0P0001FAMF.F   calma positivo    True          0.008949 Aumentar
   0P0001K6GO.F   calma positivo   False          0.020317 Mantener
   0P000125YU.L   calma positivo    True          0.013045 A